# Checking Factor Correlation and Risk Exposure

by Delaney Mackenzie

Much of the code in this notebook comes from exmamples written by Luca.

This is a quick example notebook showing how to check factor correlation. We start by computing our factors, then we construct portfolios based on the top and bottom quintiles. Then we check the correlation of returns of those portfolios.

This notebook assumes that you have a working knowledge of research and Alphalens. If you don't, check out [this tutorial](https://www.quantopian.com/tutorials/getting-started).

These cells can take a little while to run, and the time is dependent on the complexity of your factors and the length of the time window you choose.

In [ ]:
from quantopian.pipeline.factors import CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing

import numpy as np
import pandas as pd
import scipy.stats as stats

Import the Quantopian Tradeable Universe for US stocks.

In [ ]:
from quantopian.pipeline.filters import QTradableStocksUS

In [ ]:
universe = QTradableStocksUS()

Define our pipeline with the factors you want. We have some fairly simple corporate fundamentals based factors. You might want to use Custom Factors instead.

In [ ]:
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.classifiers.fundamentals import Sector 
from quantopian.pipeline.data import Fundamentals

In [ ]:
value = Fundamentals.ebit.latest / Fundamentals.enterprise_value.latest
quality = Fundamentals.roe.latest
pipe = Pipeline(
    columns = {
            'factor1' : value,
            'factor2' : quality,
            'Sector' : Sector(mask=universe), # optional, useful to compute individual sector statistics
    },
    screen=universe
)

Let's run our pipeline to get the daily values of each factor over a set time period.

Since computing factor(s) over a long period of time requires lots of memory, we use the `chunksize` argument to split our Pipeline computation into a specific number of days, limiting memory usage.

In [ ]:
factors = run_pipeline(pipe, '2013-01-01', '2014-01-01', chunksize=250) # chunksize is optional
factors = factors.dropna()

Find all the assets that were ever touched by these factors.

In [ ]:
asset_list = factors.index.levels[1]

Get pricing data for these assets.

In [ ]:
prices = get_pricing(asset_list, start_date='2013-01-01', end_date='2014-02-01', fields='open_price')

Initialize Alphalens and compute all the forward returns and how they relate to the factor values. This is the first step when running Alphalens to determine how predictive each factor is of forward returns, we'll diverge after this step.

In [ ]:
import alphalens as al

sector_labels = dict(Sector.SECTOR_NAMES)
sector_labels[-1] = "Unknown" # no dataset is perfect, better handle the unexpected

In [ ]:
factor1_data = al.utils.get_clean_factor_and_forward_returns(
    factor=factors["factor1"],
    prices=prices,
    groupby=factors["Sector"],
    quantiles=5,
    periods=(1, 5, 10)
)

factor2_data = al.utils.get_clean_factor_and_forward_returns(
    factor=factors["factor2"],
    prices=prices,
    groupby=factors["Sector"],
    quantiles=5,
    periods=(1, 5, 10)
)

Use built-in functionality to compute the returns were you invested long the highest ranked assets and short the bottom ranked assets for each factor. This is standard methodology, but keep in mind that the returns can be pretty heavily affected by the choices you make on how to construct the portfolio from the factor values.

In [ ]:
factor1_returns, factor1_positions, factor1_benchmark = \
    al.performance.create_pyfolio_input(factor1_data,
                                        period='5D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='1D')

factor2_returns, factor2_positions, factor2_benchmark = \
    al.performance.create_pyfolio_input(factor2_data,
                                        period='5D',
                                        capital=1000000,
                                        long_short=True,
                                        group_neutral=False,
                                        equal_weight=True,
                                        quantiles=[1,5],
                                        groups=None,
                                        benchmark_period='1D')

Plot the returns.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
factor1_returns.plot()
factor2_returns.plot()
plt.ylabel('Returns')
plt.legend(['Factor1', 'Factor2']);

Looks a little correlated, let's check.

In [ ]:
np.corrcoef([factor1_returns, factor2_returns])

Correlation is 0.84. This method can be extended to N factors by just adding them appropriately in the code above.

## Risk Exposure
Now we'll check the exposure of Factor 1 to the Quantopian risk model factors via Pyfolio. This needs a few pieces of information on risk factor loadings that we'll generate here. To check the exposure of Factor 2 you can just change the code to reference `factor2` instead.

In [ ]:
import pyfolio as pf

from quantopian.research.experimental import get_factor_loadings, get_factor_returns

asset_list = factor1_data.index.levels[1].unique()
start_date = factor1_data.index.levels[0].min()
end_date   = factor1_data.index.levels[0].max()

factor_loadings = get_factor_loadings(asset_list, start_date, end_date)
factor_returns = get_factor_returns(start_date, end_date)

factor_loadings.index.names = ['dt', 'ticker']

In [ ]:
pf.tears.create_perf_attrib_tear_sheet(factor1_returns,
                                       positions=factor1_positions,
                                       factor_returns=factor_returns,
                                       factor_loadings=factor_loadings,      
                                       pos_in_dollars=True)

*This presentation is for informational purposes only and does not constitute an offer to sell, a solicitation to buy, or a recommendation for any security; nor does it constitute an offer to provide investment advisory or other services by Quantopian, Inc. ("Quantopian"). Nothing contained herein constitutes investment advice or offers any opinion with respect to the suitability of any security, and any views expressed herein should not be taken as advice to buy, sell, or hold any security or as an endorsement of any security or company. In preparing the information contained herein, Quantopian, Inc. has not taken into account the investment needs, objectives, and financial circumstances of any particular investor. Any views expressed and data illustrated herein were prepared based upon information, believed to be reliable, available to Quantopian, Inc. at the time of publication. Quantopian makes no guarantees as to their accuracy or completeness. All information is subject to change and may quickly become unreliable for various reasons, including changes in market conditions or economic circumstances.*